In [1]:
using Laplacians

objc[1815]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[1815]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[1815]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[1815]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgVBLASMatNorm (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
a = readFromFile("../graphs/se_1/graph.mtx");

In [5]:
tree = readFromFile("../graphs/se_1/tree.mtx");

In [6]:
x = readFromFile("../graphs/se_1/x.vec");

In [7]:
# b = readFromFile("../graphs/se_1/b.vec");
b = lap(a) * x;

In [8]:
norm(lap(a) * x - b) / norm(b)

0.0

In [9]:
la = lap(a);

In [10]:
ts = treeSolver(tree)

F = function(b)
    return ts(b - mean(b))
end

(anonymous function)

In [13]:
@time myx64 = pcg(la, b, F, tol=1e-3, maxits=100000, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)

PCG stopped after: 60 iterations with relative error 0.000723563275623587.
  0.071634 seconds (811.76 k allocations: 34.176 MB, 10.79% gc time)


0.0007235632756237976

In [14]:
dbg = []

for precision in [64,128,1024]
    hp = precision
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    tsHP = treeSolver(treeHP)

    FHP = function(b)
        return tsHP(b - mean(b))
    end
    
    push!(dbg, "======= $(hp) bit precision")
    @time myxHP = pcgVMatNorm(laHP, bHP, FHP, xHP, tol=1e-2, maxits=100000, verbose=true);
#     @time myxHP = pcgV(laHP, bHP, FHP, tol=1e-3, maxits=100000, verbose=true);
    norm(laHP * myxHP - bHP) / norm(bHP)
    push!(dbg, "")
    push!(dbg, "")
    
end

Working on iteration 100 with error 2.57354271870849832138e+02
Working on iteration 200 with error 9.63160023344876436713e+01
Working on iteration 300 with error 5.71301616074043383496e+01
Working on iteration 400 with error 4.43248785078912704224e+01
Working on iteration 500 with error 3.56007947681082796629e+01
Working on iteration 600 with error 2.95267170897423121573e+01
Working on iteration 700 with error 2.59693377951222102457e+01
Working on iteration 800 with error 2.29197458053969821273e+01
Working on iteration 900 with error 2.09477608931427944049e+01
Working on iteration 1000 with error 1.90519844367571432528e+01
Working on iteration 1100 with error 1.70891487427716838518e+01
Working on iteration 1200 with error 1.62740423960364101515e+01
Working on iteration 1300 with error 1.5434229160104060553e+01
Working on iteration 1400 with error 1.50195097391641419742e+01
Working on iteration 1500 with error 1.46525738141191488421e+01
Working on iteration 1600 with error 1.42659582354

In [16]:
writeToFile("../graphs/se_1/_log_julia.txt", dbg)